<a href="https://colab.research.google.com/github/Sana-Saeed/Automate-Text-Pruning/blob/main/unique_review_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Installing libraries**

In [ ]:
!pip install pandas
!pip install sentence-transformers
!pip install scikit-learn


In [ ]:
!pip install contractions

# **Mount Google Drive**

In [ ]:
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Path to the CSV file on your Google Drive
csv_path = '/content/drive/MyDrive/resturent dataset/Reviews.csv'

# Read the first 20,000 rows from the CSV file


Mounted at /content/drive


# **Read Data**

In [ ]:
df = pd.read_csv(csv_path)
reviews = df["Reviews"].tolist()


# **Text Preprocessing**

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import contractions


lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))
def preprocess_text(text):
    # Remove HTML tags using BeautifulSoup
    text = BeautifulSoup(text, "html.parser").get_text()

    # Remove hyperlinks
    text = re.sub(r"http\S+", "", text)

    # Remove mentions (e.g., @username)
    text = re.sub(r"@\w+", "", text)

    # Remove punctuation
    text = re.sub(r"[^\w\s]", " ", text)

    # Expand contractions
    expanded_text = contractions.fix(text)

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]

    # Lemmatize the tokens
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    # Join tokens back into a cleaned text
    cleaned_text = " ".join(lemmatized_tokens)
    cleaned_text=cleaned_text.lower()

    return cleaned_text
preprocessed_reviews = [preprocess_text(review) for review in reviews]

unique_reviews_set = set(preprocessed_reviews)

# Convert the set back to a list
unique_reviews = list(unique_reviews_set)

<ipython-input-7-65e7fab76da7>:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


In [ ]:
print("number of reviews before removing duplicate reviews", len(preprocessed_reviews))
print("number of reviews after removing duplicate reviews",len(unique_reviews ))
df_unique= pd.DataFrame(unique_reviews, columns=['Reviews'])

number of reviews before removing duplicate reviews 20000
number of reviews after removing duplicate reviews 7657


# **Text Clustering**

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import numpy as np


# Initialize SentenceTransformer with a pre-trained BERT model
model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

# Get BERT embeddings for the reviews
embeddings = model.encode(unique_reviews, show_progress_bar=True)



Batches:   0%|          | 0/240 [00:00<?, ?it/s]

In [ ]:
# Apply KMeans clustering
num_clusters = 8
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
cluster_labels = kmeans.fit_predict(embeddings)

# Add cluster labels to the DataFrame
df_unique["Cluster"] = cluster_labels

# Store the clustering results in a new CSV file
#result_df = df[["Reviews", "Cluster"]]
df_unique.to_csv("clustering_results.csv", index=False)

print("Clustering results saved to clustering_results.csv")

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Clustering results saved to clustering_results.csv


In [ ]:
import os

# Define the path to the target folder
folder_path = "/content/drive/MyDrive/resturent dataset/"

# Create the folder if it doesn't exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

import shutil

# Define the source and destination paths
source_path = "clustering_results.csv"
destination_path = os.path.join(folder_path, "clustering_results_new.csv")

# Copy the file to the destination
shutil.copy(source_path, destination_path)

print("File copied to destination:", destination_path)


File copied to destination: /content/drive/MyDrive/resturent dataset/clustering_results_new.csv


In [ ]:
unique_clusters = df_unique["Cluster"].unique()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Create an empty DataFrame with columns
unique_rev = pd.DataFrame(columns=["Reviews", "Cluster"])

print("Processing cluster:", 1)

    # Example: Filtering and printing reviews for the current cluster value
cluster_reviews = df_unique[df_unique["Cluster"] == 1]["Reviews"]

vectorizer = TfidfVectorizer()
    # Transform the reviews into TF-IDF vectors
tfidf_matrix = vectorizer.fit_transform(cluster_reviews)
    # Calculate the cosine similarity matrix
cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)
i=0
    # Assuming you have the cosine_similarities matrix

















Processing cluster: 1
[[0 1 1 ... 1 1 1]
 [0 0 1 ... 1 1 1]
 [0 0 0 ... 1 1 1]
 ...
 [0 0 0 ... 0 1 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]]


In [ ]:
th_matrix = np.where(cosine_similarities > 0.8, 1, 0)

upper_half = np.triu(th_matrix , k=1)  # Select the upper triangular part excluding diagonal

print(upper_half)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
column_sums = np.sum(upper_half, axis=0)

print(column_sums)


[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  1  1  0  0  0  0  0  0  0  0  0  0  1  0  1  0  0  0  0  0  0  0  0  0
  2  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0
  2  0  0  1  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  1  0  0  1
  0  0  0  0  0  0  0  0  2  0  0  0  2  0  2  3  0  0  0  0  0  1  3  0
  1  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0
  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  1  0  3
  0  0  0  1  0  0  0  0  1  0  2  0  2  0  2  0  0  0  0  0  0  0  0  0
  0  1  0  0  0  0  0  0  0  0  3  0  0  0  0  1  0  0  0  0  0  0  0  0
  0  0  0  0  1  1  0  0  1  0  0  0  0  0  0  0  1  0  0  1  1  1  0  0
  0  0  0  0  0  0  1  0  0  0  0  1  0  0  0  0  0

In [ ]:

row_indices, col_indices= np.nonzero(upper_half)

print(row_indices)

print(col_indices)


[  0   0   4   7   7  10  10  15  17  17  17  17  17  17  21  21  22  22
  24  24  27  27  28  28  33  39  39  46  46  49  55  55  55  55  55  67
  67  68  71  71  72  73  73  73  73  73  73  73  75  78  78  78  82  82
  82  82  82  83  84  86  89  91  93  94  94  94  94  94  94  94  94  94
  94  94  94  96  96  96  96  96  96  96  98  98  99  99  99 102 105 106
 106 106 106 106 106 106 106 106 106 106 106 106 106 106 107 107 107 107
 108 108 109 109 109 110 116 116 116 119 125 125 127 136 138 142 142 142
 142 147 152 152 152 156 158 158 164 164 164 166 167 168 169 169 173 173
 173 173 173 173 175 175 175 175 176 176 178 182 182 182 183 183 183 183
 183 183 186 186 186 188 188 189 189 189 189 189 189 189 189 190 190 190
 190 190 191 191 191 199 203 203 205 205 206 210 213 219 220 224 228 228
 234 236 237 237 237 237 237 237 237 237 237 237 237 237 237 237 239 243
 243 248 250 250 250 250 250 250 250 250 250 250 250 250 250 254 254 254
 254 254 256 256 256 260 260 263 263 265 265 265 26

In [ ]:
nonzero_indices = np.argmax(upper_half != 0, axis=1)

print(nonzero_indices)




[224   0   0   0 248   0   0 105   0   0 376   0   0   0   0 784   0  73
   0   0   0  86  72   0 210   0   0 308 547   0   0   0   0 348   0   0
   0   0   0  84   0   0   0   0   0   0 147   0   0 615   0   0   0   0
   0 127   0   0   0   0   0   0   0   0   0   0   0 495 182   0   0 524
 659  96   0 446   0   0 433   0   0   0 394 750 144   0 691   0   0 639
   0 522   0 654 189   0 183   0 318 164   0   0 192   0   0 705 237 351
 199 557 167   0   0   0   0   0 333   0   0 182   0   0   0   0   0 362
   0 476   0   0   0   0   0   0   0   0 464   0 490   0   0   0 190   0
   0   0   0 180   0   0   0   0 417   0   0   0 476   0 547   0   0   0
   0   0 176   0 309 428 348 279   0   0   0 436   0 307 239   0 243   0
   0   0 323 190   0   0 490   0 461 254 365 460   0   0   0   0   0   0
   0 252   0   0   0 293   0 296 695   0   0   0 596   0   0 409   0   0
   0   0   0 513 721   0   0   0 603   0   0   0 394   0   0   0   0   0
 530   0 364 250   0 378   0   0   0 800   0   0   

In [ ]:
print(len(nonzero_indices))
print(len(row_indices))
print(len(col_indices))

905
497
497


In [ ]:
non_zero_list = list(filter(lambda x: x != 0, nonzero_indices))

print(non_zero_list)
print(len(non_zero_list))

[224, 248, 105, 376, 784, 73, 86, 72, 210, 308, 547, 348, 84, 147, 615, 127, 495, 182, 524, 659, 96, 446, 433, 394, 750, 144, 691, 639, 522, 654, 189, 183, 318, 164, 192, 705, 237, 351, 199, 557, 167, 333, 182, 362, 476, 464, 490, 190, 180, 417, 476, 547, 176, 309, 428, 348, 279, 436, 307, 239, 243, 323, 190, 490, 461, 254, 365, 460, 252, 293, 296, 695, 596, 409, 513, 721, 603, 394, 530, 364, 250, 378, 800, 834, 274, 361, 304, 292, 503, 651, 538, 651, 334, 413, 376, 520, 495, 439, 731, 353, 467, 366, 645, 363, 747, 534, 892, 549, 497, 428, 581, 617, 663, 669, 611, 455, 425, 385, 660, 695, 593, 839, 475, 701, 803, 709, 626, 703, 528, 695, 888, 555, 475, 624, 758, 538, 759, 550, 538, 547, 732, 637, 775, 695, 739, 654, 869, 871, 793, 558, 661, 587, 576, 609, 608, 647, 703, 780, 846, 693, 605, 822, 824, 611, 654, 645, 695, 842, 703, 658, 695, 901, 683, 685, 742, 681, 838, 710, 838, 864, 776, 779, 756, 736, 892, 765, 880, 790, 880, 853, 799, 822, 804, 875, 805, 835, 845, 880, 877, 868, 879]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# Create an empty DataFrame with columns
unique_rev = pd.DataFrame(columns=["Reviews", "Cluster"])




# Assuming you have the list unique_clusters

for cluster_value in unique_clusters:
    # Perform actions for each unique cluster value
    print("Processing cluster:", cluster_value)

    # Example: Filtering and printing reviews for the current cluster value
    cluster_reviews = result_df[result_df["Cluster"] == cluster_value]["Reviews"]

    vectorizer = TfidfVectorizer()
    # Transform the reviews into TF-IDF vectors
    tfidf_matrix = vectorizer.fit_transform(cluster_reviews)
    # Calculate the cosine similarity matrix
    cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)
    i=0
    # Assuming you have the cosine_similarities matrix
    num_documents = cosine_similarities.shape[0]
    for i, c in  zip(range(num_documents), cluster_reviews):
        for j in range(i + 1, num_documents):  # Start from i + 1 to exclude diagonal

            if cosine_similarities[i][j]<80:
              unique_rev = unique_rev.append({"Reviews": c, "Cluster": cluster_value}, ignore_index=True)















#for i, c in  zip(range(num_documents), cluster_reviews):
 #   for j in range(i + 1, j):  # Start from i + 1 to exclude diagonal
  #    print(c)













Processing cluster: 15


Streaming output truncated to the last 5000 lines.
<ipython-input-44-34fb286d12f0>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unique_rev = unique_rev.append({"Reviews": c, "Cluster": cluster_value}, ignore_index=True)
<ipython-input-44-34fb286d12f0>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unique_rev = unique_rev.append({"Reviews": c, "Cluster": cluster_value}, ignore_index=True)
<ipython-input-44-34fb286d12f0>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unique_rev = unique_rev.append({"Reviews": c, "Cluster": cluster_value}, ignore_index=True)
<ipython-input-44-34fb286d12f0>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat 

Processing cluster: 4


Streaming output truncated to the last 5000 lines.
<ipython-input-44-34fb286d12f0>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unique_rev = unique_rev.append({"Reviews": c, "Cluster": cluster_value}, ignore_index=True)
<ipython-input-44-34fb286d12f0>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unique_rev = unique_rev.append({"Reviews": c, "Cluster": cluster_value}, ignore_index=True)
<ipython-input-44-34fb286d12f0>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unique_rev = unique_rev.append({"Reviews": c, "Cluster": cluster_value}, ignore_index=True)
<ipython-input-44-34fb286d12f0>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat 

Processing cluster: 6


Streaming output truncated to the last 5000 lines.
<ipython-input-44-34fb286d12f0>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unique_rev = unique_rev.append({"Reviews": c, "Cluster": cluster_value}, ignore_index=True)
<ipython-input-44-34fb286d12f0>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unique_rev = unique_rev.append({"Reviews": c, "Cluster": cluster_value}, ignore_index=True)
<ipython-input-44-34fb286d12f0>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unique_rev = unique_rev.append({"Reviews": c, "Cluster": cluster_value}, ignore_index=True)
<ipython-input-44-34fb286d12f0>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat 

In [ ]:
import numpy as np

matrix = np.array([
    [0.8, 0.5, 0.9],
    [0.3, 0.6, 0.2],
    [0.7, 0.4, 0.1]
])

result_matrix = np.where(matrix > 0.7, 0, 1)

print(result_matrix)


[[0 1 0]
 [1 1 1]
 [1 1 1]]


In [ ]:
import numpy as np

matrix = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])

upper_half = np.triu(matrix, k=1)  # Select the upper triangular part excluding diagonal

print(upper_half)

[[0 2 3]
 [0 0 6]
 [0 0 0]]
